In [ ]:
# Базовые библиотеки и утилиты Python
import logging
import os
from threading import Thread
from typing import List, Optional

# Обработка и анализ данных
import pandas as pd

# Библиотеки машинного обучения и моделирования
import mlflow
from mlflow.models.signature import infer_signature
from mlflow.pyfunc import PythonModel
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# Работа с облачными сервисами
import boto3
from botocore.client import Config

# Визуализация и интерактивность
from tqdm import notebook_tqdm as tqdm

# Локальные модули проекта (связанные с чат-ботами и их компонентами)
from langchain.chat_models.base import BaseChatModel
from langchain.schema import AIMessage, BaseMessage, ChatGeneration, ChatResult, HumanMessage, SystemMessage

# Импортирование модулей сервиса
from ConversationManager import ConversationManager


class VicunaBot(mlflow.pyfunc.PythonModel):
    """
    Класс для управления чат-диалогом с помощью LLM.

    Описание: Управляет взаимодействием между пользователем и моделью LLM, генерирует подсказки и ответы.
    """
    def __init__(self, model: LlamaForCausalLM, tokenizer: LlamaTokenizer, device='cuda': str, gen_kwargs: dict):

        self.tokenizer = tokenizer
        self.model = model
        self.device = device
        self.gen_kwargs = gen_kwargs

    def generate(self, messages: List[BaseMessage], stop: Optional[List[str]] = None, promt: str) -> ChatResult:
        """
        Генерация ответа модели LLM.

        Описание: Генерирует ответ на основе текущего контекста и подсказки.
        """

        from langchain.schema import AIMessage

        inputs = self.tokenizer(prompt, return_tensors='pt')

        outputs = self.model.generate(inputs.input_ids.to(self.device), **self.gen_kwargs)
        generated_text = self.tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]

        return AIMessage(content=generated_text.strip())

    def a_generate(self) -> None:
        return None

    @property
    def llm_type(self) -> str:
        return "custom"

    def predict(self, context, model_input):
        """
        Вычисление предсказаний модели на основе входных данных и сохранение диалога.

        Параметры:
            context: Не используется в текущей реализации, предназначен для будущих расширений.
            model_input (DataFrame): DataFrame содержащий данные пользователя, включая user_id и user_input.

        Возвращает:
            DataFrame: DataFrame с ответом ассистента, содержащий колонки user_id и assistant_answer.

        Описание:
            Метод обрабатывает запросы от пользователя, генерирует ответы с использованием модели LLM
            и вызывает методы dump_conversation или update_conversation для сохранения или обновления
            истории диалога в базу данных.
        """
        # Проверка готовности модели
        if not self.is_model_ready:
            return {'status': 'model not ready'}

        # Инициализация или обновление контекста пользователя
        user_id = model_input.user_id[0]
        user_input = model_input.user_input[0]
        self.context = ' '
        href = ' '

        if user_id in self.user_ids:
            self.user_ids[user_id][1] = user_input
            update_history = True  # Обновление существующего диалога
        else:
            self.user_ids[user_id] = self.user_init()
            self.user_ids[user_id][1] = user_input
            update_history = False  # Новый диалог

        # Генерация ответа ассистента
        assistant_response = self.get_assistant_response()

        # Разбиение контекста на содержание и ссылку, если есть
        if 'Подробнее про это можно прочитать тут:' in self.context:
            cont, href = self.context.split('Подробнее про это можно прочитать тут: ')

        # Добавление ссылки в ответ, если необходимо
        if href and href not in assistant_response.content and not self.context_flag:
            assistant_response.content += ' Подробнее об этом можно прочитать тут:' + href

        # Логирование результата
        logging.info(f'\n___________________________________\nid":{user_id},\n___________________________________\n"resp":{assistant_response.content},\n___________________________________\n"prompt":{self.prompt}\n___________________________________\n')

        return pd.DataFrame({
            'user_id': [user_id],
            'assistant_answer': [assistant_response.content]
        })


    def get_assistant_response(self):
        """
        Получение ответа от ассистента.

        Описание: Генерирует ответы на запросы пользователей, основываясь на текущем контексте диалога.
        """
        from langchain.schema import BaseMessage, AIMessage, HumanMessage, SystemMessage, ChatResult, ChatGeneration

        chat_model, user_input, dialogue_history = self.user_ids[self.id]
        user_message = HumanMessage(content=user_input)
        assistant_responses = dialogue_history.get_assistant_responses()   # Get the list of assistant responses as strings
        assistant_response = chat_model.generate([user_message], dialogue_history, assistant_responses)
        dialogue_history.add(user_input, assistant_response)
        return assistant_response # Pass the list of strings as dialogue_history

    def save_model(model_name, registered_model_name):
        """
        Сохранение модели в MLflow.

        Описание: Регистрирует и сохраняет модель в системе управления версиями MLflow.
        """
        import pandas as pd
        from mlflow.models.signature import infer_signature
        mlflow.set_tracking_uri("http://mlflow")
        with mlflow.start_run(experiment_id=6) as run:
            # Define the path to save your model artifacts
            # filename = 'prod/vicuna_bot/model'

            # Log any additional artifacts
            # mlflow.log_artifact('model', artifact_path='model')
            # mlflow.log_artifact('convs.db', artifact_path='convs.db')
            # mlflow.log_artifact('texts.docx', artifact_path= 'texts.docx')
            artifacts = {"doc_path": 'texts.docx'}

            # Define the model signature based on input and output data
            input_schema = pd.DataFrame(data=[[2,'qweqwe', 1]], columns=['user_id', 'user_input', 'sys_flag' ])
            output_schema = pd.DataFrame(data=[[2,'qweqwe']], columns=['user_id', 'assistent_answer'])

            signature = infer_signature(input_schema, output_schema)

            # Log the model using mlflow.pyfunc
            mlflow.pyfunc.log_model(
                artifact_path='model',
                python_model=VicunaBot(),
                artifacts=artifacts,
                pip_requirements="requirements_vicuna.txt",
                signature=signature,
                registered_model_name=registered_model_name
            )

            # Get the model URI
            #model_uri = mlflow.get_artifact_uri(filename)

        return None #model_uri